In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/donors-choose/preprocessed_data.csv
/kaggle/input/glove6b100dtxt/glove.6B.100d.txt


In [2]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

In [3]:
embeddings_index = dict()
f = open('/kaggle/input/glove6b100dtxt/glove.6B.100d.txt',encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:])
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [4]:
data=pd.read_csv('/kaggle/input/donors-choose/preprocessed_data.csv')

In [5]:
y=data['project_is_approved']

In [6]:
data=data.drop(['project_is_approved'],axis=1)

In [7]:
preproc=[]
for row in data['project_grade_category']:
    row=row.replace('grades','')
    row=row.replace('_prek_2','prek2')
    row=row.replace('_3_5','3to5')
    row=row.replace('_6_8','6to8')
    row=row.replace('_9_12','9to12')
    preproc.append(row)
    
data['project_grade_category']=preproc

In [8]:
preproc=[]
for row in data['clean_categories']:
    row=row.replace(' ','')
    row=row.replace('_','')
    preproc.append(row)
    
data['clean_categories']=preproc

In [9]:
preproc=[]
for row in data['clean_subcategories']:
    row=row.replace(' ','')
    row=row.replace('_','')
    preproc.append(row)
    
data['clean_subcategories']=preproc

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.33, stratify=y)

In [12]:
X_train.columns

Index(['school_state', 'teacher_prefix', 'project_grade_category',
       'teacher_number_of_previously_posted_projects', 'clean_categories',
       'clean_subcategories', 'essay', 'price'],
      dtype='object')

In [13]:
from keras.models import Sequential,Model
from keras.layers import Dense,BatchNormalization
from keras.preprocessing.text import Tokenizer
from keras.layers import LSTM,Input,Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.optimizers import Adam
from keras.preprocessing.sequence import pad_sequences
from keras.layers import BatchNormalization
from keras.initializers import he_normal

Using TensorFlow backend.


# Model 3

### Essay

In [14]:
#credit to https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/
token = Tokenizer()
token.fit_on_texts(X_train['essay'])
vocabulary_size = len(token.word_index)+1

In [15]:
embedding_matrix = np.zeros((vocabulary_size, 100))
for word, i in token.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [16]:
word_count = []

for e in data["essay"] :
    c = len(e.split())
    word_count.append(c)

In [17]:
max_word_count=max(word_count)

In [18]:
max_word_count

339

In [19]:
train_essay=token.texts_to_sequences(X_train['essay'])
test_essay=token.texts_to_sequences(X_test['essay'])

In [20]:
train_essay=pad_sequences(train_essay,maxlen=350)
test_essay=pad_sequences(test_essay,maxlen=350)

In [21]:
embedding_matrix.shape

(48181, 100)

In [22]:
max_length=350
input_essay = Input(shape=(max_length,))
embedding = Embedding(vocabulary_size, 100,weights=[embedding_matrix], input_length=max_length,trainable=False)(input_essay)
lstm=LSTM(100,return_sequences=True)(embedding)
flatten=Flatten()(lstm)

### Remaining Numerical Features

In [23]:
from scipy.sparse import hstack

train_price=X_train['price'].values.reshape(-1, 1)
train_number=X_train['teacher_number_of_previously_posted_projects'].values.reshape(-1, 1)

test_price=X_test['price'].values.reshape(-1, 1)
test_number=X_test['teacher_number_of_previously_posted_projects'].values.reshape(-1, 1)

rem_train=np.concatenate((train_price,train_number),axis=1)
rem_test=np.concatenate((test_price,test_number),axis=1)

In [24]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
scaler.fit(rem_train)
rem_train_standard=scaler.transform(rem_train)
rem_test_standard=scaler.transform(rem_test)

In [25]:
rem_feat = Input(shape=(2,))
rem_feat_dense= Dense(128, activation='relu')(rem_feat)

## One Hot Encoding of Categorical Features

### School State

In [26]:
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer

In [27]:
my_counter=Counter()
for word in data['school_state'].values:
    my_counter.update(word.split())
vocab_dict=dict(my_counter)
sorted_vocab_dict=dict(sorted(vocab_dict.items(),key=lambda kv:kv[1]))

In [28]:
vectorizer=CountVectorizer(vocabulary=list(sorted_vocab_dict.keys()),lowercase=False,binary=True)
vectorizer.fit(X_train['school_state'].values)

CountVectorizer(analyzer='word', binary=True, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=False, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None,
                vocabulary=['vt', 'wy', 'nd', 'mt', 'ri', 'sd', 'ne', 'de',
                            'ak', 'nh', 'wv', 'me', 'hi', 'dc', 'nm', 'ks',
                            'ia', 'id', 'ar', 'co', 'mn', 'or', 'ky', 'ms',
                            'nv', 'md', 'ct', 'tn', 'ut', 'al', ...])

In [29]:
train_state_ohe=vectorizer.transform(X_train['school_state'].values)
test_state_ohe=vectorizer.transform(X_test['school_state'].values)

### Project Grade Category

In [30]:
my_counter=Counter()
for word in data['project_grade_category'].values:
    my_counter.update(word.split())
vocab_dict=dict(my_counter)
sorted_vocab_dict=dict(sorted(vocab_dict.items(),key=lambda kv:kv[1]))

In [31]:
vectorizer=CountVectorizer(vocabulary=list(sorted_vocab_dict.keys()),lowercase=False,binary=True)
vectorizer.fit(X_train['project_grade_category'].values)
train_grade_ohe=vectorizer.transform(X_train['project_grade_category'].values)
test_grade_ohe=vectorizer.transform(X_test['project_grade_category'].values)

### Clean Categories

In [32]:
my_counter=Counter()
for word in data['clean_categories'].values:
    my_counter.update(word.split())
vocab_dict=dict(my_counter)
sorted_vocab_dict=dict(sorted(vocab_dict.items(),key=lambda kv:kv[1]))

In [33]:
vectorizer=CountVectorizer(vocabulary=list(sorted_vocab_dict.keys()),lowercase=False,binary=True)
vectorizer.fit(X_train['clean_categories'].values)
train_cat_ohe=vectorizer.transform(X_train['clean_categories'].values)
test_cat_ohe=vectorizer.transform(X_test['clean_categories'].values)

### Clean Subcategories

In [34]:
my_counter=Counter()
for word in data['clean_subcategories'].values:
    my_counter.update(word.split())
vocab_dict=dict(my_counter)
sorted_vocab_dict=dict(sorted(vocab_dict.items(),key=lambda kv:kv[1]))

In [35]:
vectorizer=CountVectorizer(vocabulary=list(sorted_vocab_dict.keys()),lowercase=False,binary=True)
vectorizer.fit(X_train['clean_subcategories'].values)
train_sub_ohe=vectorizer.transform(X_train['clean_subcategories'].values)
test_sub_ohe=vectorizer.transform(X_test['clean_subcategories'].values)

### Teacher Prefix

In [36]:
my_counter=Counter()
for word in data['teacher_prefix'].values:
    my_counter.update(word.split())
vocab_dict=dict(my_counter)
sorted_vocab_dict=dict(sorted(vocab_dict.items(),key=lambda kv:kv[1]))

In [37]:
vectorizer=CountVectorizer(vocabulary=list(sorted_vocab_dict.keys()),lowercase=False,binary=True)
vectorizer.fit(X_train['teacher_prefix'].values)
train_prefix_ohe=vectorizer.transform(X_train['teacher_prefix'].values)
test_prefix_ohe=vectorizer.transform(X_test['teacher_prefix'].values)

In [38]:
import scipy

In [39]:
train_data=scipy.sparse.hstack((train_state_ohe,train_grade_ohe,train_cat_ohe,train_sub_ohe,train_prefix_ohe,rem_train_standard)).tocsr().todense()
test_data=scipy.sparse.hstack(((test_state_ohe),(test_grade_ohe),(test_cat_ohe),(test_sub_ohe),(test_prefix_ohe),(rem_test_standard))).tocsr().todense()

In [40]:
train_data=np.expand_dims(train_data, axis=2)
test_data=np.expand_dims(test_data, axis=2)

In [43]:
# https://github.com/pranaya-mathur/Donors-Choose-LSTM/blob/master/DonorsChoose_Model_1_13_Aug_19.ipynb
import tensorflow as tf
from sklearn.metrics import roc_auc_score

def auc_score(y_true, y_pred):
    if len(np.unique(y_true[:])) == 1:
        return 0.5
    else:
        return roc_auc_score(y_true, y_pred)

def auc_sc(y_true, y_pred):
    return tf.py_func(auc_score, (y_true, y_pred), tf.double)   

In [44]:
from keras.utils import np_utils
Y_train=np_utils.to_categorical(y_train)
Y_test=np_utils.to_categorical(y_test)

In [48]:
from keras.layers import Conv1D
from keras.layers import MaxPooling1D,Dropout,concatenate

input_conv =  Input(shape=(train_data.shape[1],1))
Layer1=Conv1D(filters=96,kernel_size=3,activation='relu')(input_conv)
Layer2=MaxPooling1D(3,padding='same')(Layer1)
Layer3=Dropout(0.5)(Layer2)

Layer4=Conv1D(filters=128,kernel_size=3,activation='relu',padding='same')(Layer3)
Layer5=MaxPooling1D(3,padding='same')(Layer4)
Layer6=Dropout(0.5)(Layer5)

Layer7=Flatten()(Layer6)

In [58]:
concat = concatenate([flatten,Layer7])

Layer8 = Dense(456, activation='relu',kernel_initializer=he_normal())(concat)
Layer9=Dropout(0.5)(Layer8)

Layer10= Dense(256, activation='relu',kernel_initializer=he_normal())(Layer9)
Layer11=Dropout(0.5)(Layer10)

Layer12 = Dense(128, activation='relu',kernel_initializer=he_normal())(Layer11)
Layer13=BatchNormalization()(Layer12)

main_output1 = Dense(2, activation='softmax')(Layer13)

In [59]:
model3 = Model(inputs=[input_essay,input_conv], outputs=[main_output1])

In [60]:
model3.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 514, 1)       0                                            
__________________________________________________________________________________________________
conv1d_4 (Conv1D)               (None, 512, 96)      384         input_5[0][0]                    
__________________________________________________________________________________________________
max_pooling1d_4 (MaxPooling1D)  (None, 171, 96)      0           conv1d_4[0][0]                   
__________________________________________________________________________________________________
dropout_3 (Dropout)             (None, 171, 96)      0           max_pooling1d_4[0][0]            
____________________________________________________________________________________________

In [61]:
import tensorflow as tf
from keras.callbacks import TensorBoard,EarlyStopping
from time import time

earlystop_1 = EarlyStopping(monitor = 'val_loss', mode="max",min_delta = 0, patience = 4,verbose = 1,restore_best_weights = True)
tensorboard_1 = TensorBoard("logs3")

In [62]:
model3.compile(optimizer='adam', loss='binary_crossentropy', metrics=[auc_sc])

In [63]:
train_data3=[train_essay,train_data]
test_data3=[test_essay,test_data]

In [64]:
model3.fit(train_data3,Y_train,epochs=8, batch_size=64,verbose=1,validation_data=(test_data3, Y_test),class_weight='balanced',callbacks=[tensorboard_1,earlystop_1])

Train on 73196 samples, validate on 36052 samples
Epoch 1/8
73196/73196 [==============================] - 470s 6ms/step - loss: 0.4393 - auc_sc: 0.6031 - val_loss: 0.4122 - val_auc_sc: 0.7138
Epoch 2/8
73196/73196 [==============================] - 466s 6ms/step - loss: 0.3863 - auc_sc: 0.7175 - val_loss: 0.3777 - val_auc_sc: 0.7352
Epoch 3/8
73196/73196 [==============================] - 468s 6ms/step - loss: 0.3748 - auc_sc: 0.7413 - val_loss: 0.3744 - val_auc_sc: 0.7451
Epoch 4/8
73196/73196 [==============================] - 470s 6ms/step - loss: 0.3665 - auc_sc: 0.7574 - val_loss: 0.3750 - val_auc_sc: 0.7499
Epoch 5/8
73196/73196 [==============================] - 468s 6ms/step - loss: 0.3542 - auc_sc: 0.7778 - val_loss: 0.3708 - val_auc_sc: 0.7498
Restoring model weights from the end of the best epoch
Epoch 00005: early stopping


In [1]:
from prettytable import PrettyTable
x = PrettyTable()
x.field_names = ["Model", "train_auc", "test_auc"]
x.add_row(["Model_3", 0.7778,0.7498])
print(x)

+---------+-----------+----------+
|  Model  | train_auc | test_auc |
+---------+-----------+----------+
| Model_3 |   0.7778  |  0.7498  |
+---------+-----------+----------+
